In [1]:
import sys
sys.path.insert(0, 'C:\\Users\\Paarth Tandon\\Desktop\\repos\\personal-music-gen')

In [2]:
from audiocraft.models import MusicGen
from audiocraft.utils.notebook import display_audio
from personal_musicgen.model_utils import generate

import torch
import torchaudio

A matching Triton is not available, some optimizations will not be enabled.
Error caught was: No module named 'triton'
c:\Users\Paarth Tandon\Desktop\repos\personal-music-gen\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model = MusicGen.get_pretrained('small')
model.lm = model.lm.to(torch.float32)
model.lm.load_state_dict(torch.load('../checkpoints/xs5kf1lg/checkpoint_949.pth')['model_state_dict'])

c:\users\paarth tandon\desktop\repos\personal-music-gen\libs\audiocraft\audiocraft\models\musicgen.py:116: UserWarning: MusicGen pretrained model relying on deprecated checkpoint mapping. Please use full pre-trained id instead: facebook/musicgen-small
  warnings.warn(
c:\Users\Paarth Tandon\Desktop\repos\personal-music-gen\.venv\lib\site-packages\torch\nn\utils\weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


<All keys matched successfully>

In [4]:
import torch
from audiocraft.utils.autocast import TorchAutocast
from audiocraft.modules.conditioners import ClassifierFreeGuidanceDropout
import numpy as np
import IPython.display as ipd
from scipy.io.wavfile import write


def generate_long_seq(
    model, description, num_samples, total_gen_len, use_sampling=True, temp=1.0, top_k=250, top_p=0.0, cfg_coef=None
):
    """Instead of using a text prompt, half the sample length of the previous generation is used as prompt input 

    Args:
        model (MusicGen): The pretrained MusicGen Model
        num_samples (integer): How many samples should be created, one sample is half the totgal_gen_len
        total_gen_len (integer): Maximum generation length.
        use_sampling (bool): Whether to use a sampling strategy or not.
        temp (float): Softmax temperature parameter. Defaults to 1.0.
        top_k (integer): top_k used for sampling. Defaults to 250.
        top_p (float): top_p used for sampling, when set to 0 top_k is used. Defaults to 0.0.
        cfg_coef (float): Coefficient used for classifier free guidance. Defaults to 3.0.

    Returns:
        List[torch.Tensor]: Output is a list of numpy vectors
    """
    samples = []
    prev_generated = None
    descriptions = [description]

    attributes, prompt_tokens = model._prepare_tokens_and_attributes(descriptions, None)
    remove_prompts = False

    with model.autocast:
        for i in range(num_samples):
            sample = model.lm.generate(
                prev_generated,
                attributes,
                1,
                total_gen_len,
                use_sampling,
                temp,
                top_k,
                top_p,
                cfg_coef,
                remove_prompts=remove_prompts,
            )
            remove_prompts = True

            if sample.shape[2] == total_gen_len:
                prev_generated = torch.clone(sample[..., total_gen_len // 2 :])
            else:
                prev_generated = torch.clone(sample)

            with torch.no_grad():
                gen_audio = model.compression_model.decode(sample, None)
            # free gpu
            del sample

            gen_audio = gen_audio[0].detach().cpu().numpy()
            gen_audio = gen_audio.transpose(1, 0)
            samples.append(gen_audio)

    return samples

def display_long_audio(samples, path: str = None):
    audio = np.concatenate(samples, axis=0)
    audio = np.squeeze(audio)
    ipd.display(ipd.Audio(audio, rate=32000))

    if path:
        write(path, 32000, audio.astype(np.float32))

In [19]:
# GTZAN genres: blues, classical, country, disco, hip-hop, jazz, metal, pop, reggae, and rock

prompt = 'personal'
gen_audio = generate(model, prompt, 2)

100%|██████████| 2/2 [00:52<00:00, 26.25s/it]


In [30]:
display_audio(gen_audio, path="audioSamples.wav")

TypeError: display_audio() got an unexpected keyword argument 'path'

In [6]:
for i in range(6, 11):
    gen = generate(model, 'personal', 1)
    torchaudio.save(f'{i}.wav', gen[0], 32000)

100%|██████████| 1/1 [00:34<00:00, 34.36s/it]


In [18]:
# torchaudio.save(f'{prompt.split(",")[1].strip()}-{1}.wav', gen_audio[0], 32000)
torchaudio.save(f'5.wav', gen_audio[0], 32000)

In [53]:
prompt = 'personal'
long_gen_audio = generate_long_seq(model, prompt, 9, 512)

: 

In [52]:
display_long_audio(long_gen_audio)